In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Loading Packages**

In [12]:
import plotly.express as px


**Importing the data set for India**

In [13]:
f = open(r"../input/youtube-new/IN_category_id.json")
india_cate = json.load(f)
india_videos = pd.read_csv(r"../input/youtube-new/INvideos.csv")

In [14]:
india_videos.head(10)

# Top ten trending channels

In [15]:
def top_trending_channels(data):
    trending_channels = data.groupby(['channel_title']).sum().sort_values('views',ascending = False).head(10)
    fig1 = px.bar(trending_channels,x = trending_channels.index,y = 'views',title = 'Top Trending Channels',width = 800, height = 400)
    fig1.show()

In [16]:
top_trending_channels(india_videos)

In [17]:
def channels_with_more_trends(data):
    s = data.value_counts('channel_title').head(10)
    tot_views = data.groupby(['channel_title']).sum()
    views_vs_channel = {}
    for j in s.index.values:
        for i in range(0,len(tot_views)):
            if tot_views.index[i] == j:
                views_vs_channel[j] = tot_views.iloc[i]['views']
    rows = views_vs_channel.keys()
    columns = views_vs_channel.values()
    data = {'channel':rows,'views':columns}
    views_vs_channel_df = pd.DataFrame(data)
    fig2 = px.bar(views_vs_channel_df.sort_values('views',ascending = False),x = 'channel', y = 'views',title = 'Views of Channels with most number of trends',height = 800,width = 400)
    fig2.show()

In [18]:
channels_with_more_trends(india_videos)

# Top channels by category

In [26]:
#Adding the category data from the json file to the videos_csv file.
def getList(dict):
    return [*dict]
def add_category_data(data):
    holder = []
    for key in india_cate.items():
        holder.append(key)
    id_title_dict = {}
    for x  in holder[2]:
        for i in x[5:]:
            id_title_dict[int(i['id'])] = i['snippet']['title']
    category = []
    ids = getList(id_title_dict.keys())
    cat_names = getList(id_title_dict.values())
    for i in range(0,len(india_videos)):
        obs_id = india_videos.iloc[i]['category_id'] 
        if obs_id in ids:
            category.append(id_title_dict[obs_id])
        else:
            category.append('Unknown')
    videos_copy = data.copy(deep = True)
    videos_copy['Category'] = category
    videos_copy[videos_copy['Category'] == 'Unknown'].shape
    return videos_copy,cat_names

In [27]:
videos_with_cat,cat_names = add_category_data(india_videos)

In [28]:
cat_names

In [39]:
videos_with_cat['Category']

In [89]:
videos_with_cat

In [105]:
#Creating a seperate functions for plotting as plotly has no way to update graph inside for loop
#def plot_(data):
   # fig = px.bar(data,x = s.index, y = s.values,title = i,labels = {'y':"Number of times trended"},width = 800,height =400 )
   # fig.show()
def top_channels_categoryWise(data,cat_names):
    for i in cat_names:
        cat_data = data[data['Category'] == i]
        cat_data = cat_data['channel_title'].value_counts().head(10).to_frame('Times_trended')
        #print(cat_data.head(5))
        fig2 = px.bar(cat_data,x = cat_data.index, y = 'Times_trended',title = i,labels = {'y':"Number of times trended"},width = 800,height =400 )
        fig2.show()
        
    

In [107]:
top_channels_categoryWise(videos_with_cat,cat_names)